In [ ]:
from tqdm import tqdm
tqdm.pandas()
from glob import glob

import json
import csv
import numpy as np
import pandas as pd
from collections import Counter

import rdflib
from rdflib import Graph
from data.data import CollectionAccessor, ImageHandler

from search import Search, Randomiser

In [ ]:
def init_DMG(images_path):
    if images_path is not None:
        image_handler = ImageHandler(image_folder=images_path, keep_prefix=True)
    else:
        image_handler = None

    time_stamp, pub_file, priv_file = CollectionAccessor.get_latest_dump("./data/dumps")
    # print(CollectionAccessor.get_latest_dump("./data/dumps"))

    
    dmg_meta = dict(name="Design Museum Gent (public & private)", id_="DMG_"+time_stamp,
                creation_timestamp=time_stamp)
    df = CollectionAccessor.get_DMG(pub_path=pub_file, #get_latest("./data/dumps", contains="public"),
                                     priv_path=priv_file, #get_latest("./data/dumps", contains="private"),
                                     rights_path="./data/rights.csv",
                                     image_handler=image_handler,
                                     **dmg_meta)
    return image_handler, df

image_handler, dmg = init_DMG("./data/images/")

In [ ]:
dmg.image_path.fillna(False).apply(lambda ls: (ls[0] if ls else None))

In [ ]:
def init_DMG():
    image_folder = "./data/images/DMG"
    image_handler = ImageHandler(image_folder=image_folder, keep_prefix=False)
    print("image paths loaded...")

    time_stamp, pub_file, priv_file = CollectionAccessor.get_latest_dump("./data/dumps")
    print(CollectionAccessor.get_latest_dump("./data/dumps"))

    
    dmg_meta = dict(name="Design Museum Gent (public & private)", id_="DMG_"+time_stamp,
                creation_timestamp=time_stamp)
    df = CollectionAccessor.get_DMG(pub_path=pub_file, #get_latest("./data/dumps", contains="public"),
                                     priv_path=priv_file, #get_latest("./data/dumps", contains="private"),
                                     rights_path="./data/rights.csv",
                                     image_handler=image_handler,
                                     **dmg_meta)
    
    rand = Randomiser(df, name="Randomiser")
    rand2 = Randomiser(df, name="Randomiser")
    s = Search([rand, rand2])
    return image_handler, df, s

im, dmg, dmg_search = init_DMG()

In [ ]:
recs = dmg.sample(4)
searcher_ids = [s.id for s in dmg_search.searchers[:1]]

In [ ]:
dmg_search(recs, searcher_ids=searcher_ids)

---
# DEV FUNCTIONS

In [ ]:
def search_collection(collection_id, object_ids, concept, model_list):
    cur_coll = get_collection(collection_id)
    object_ids = parse_id_list(object_ids)
    model_list = parse_id_list(model_list)
    cur_search = searches[collection_id]
    print(object_ids)
    scores = cur_search(object_ids)


object_ids = "1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"
object_ids = parse_id_list(object_ids)
object_ids
recs = dmg.loc[object_ids]

In [ ]:
# searcher_scores = [cur_s(recs) for cur_s in dmg_search.searchers]
# searcher_scores = pd.DataFrame({cur_s.name: cur_s for cur_s in searcher_scores})
# searcher_scores.loc[recs.index] = 0.


# searcher_scores.round(3)

In [ ]:


dmg.coll.filter("doos")

---
# TESTING ROUTES

In [ ]:
!curl "http://0.0.0.0:8080/collections"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-13"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-13/object-details?object_ids=1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-13/models"

In [ ]:
!curl "http://0.0.0.0:8080/moon?ISO_8601_datetime=2024-05-13&lat_degrees=51.05&long_degrees=3.71"

In [ ]:
# default params for search
!curl "http://0.0.0.0:8080/DMG_2025-05-13/search"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-13/search?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="schreeuw"&model_list="""

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-13/search/order?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="hello"&model_list="a,b"&skip=200&limit=100&reverse=true"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-13/search/sample?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="hello"&model_list="a,b"&k=12"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-13/search/order/filter?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="hello"&model_ids=""&filter_text="doosje""

---

# routes 2

In [1]:
import requests as rq
from time import time
from tqdm import tqdm
import pandas as pd

base = "http://0.0.0.0:8080/"
# collection_id="DMG_2025-05-13/"

In [2]:
path = "search"
scores = []
for d in rq.get(base+"/collections").json():
    collection_id = d["id"] +"/"
    params = dict(object_ids="", concept="", model_ids="") #,".join(models))
    search_res = rq.get(base+collection_id+path, params).json()
    scores.append(pd.Series(search_res).sort_values())
    # score_ls.append(scores)

    # print(rq.get(base+coll_id+"/models").json())
    

In [3]:
path = "search/order"
models = rq.get(base+collection_id+"models").json()
models = [d["id"] for d in models]
params = dict(object_ids="IE1896.152,O1912.472,1984.496,O1911.243", concept="", model_ids=",".join(models))
search_res = rq.get(base+collection_id+path, params).json()
# pd.Series(search_res).hist()

In [4]:
pd.DataFrame.from_records(search_res)

,inventory_number,title,description,designer,producer,design_date,production_date,design_place,production_place,rights_attribution,image_path
0,EG1989.1.128,Frau in langem Gewand mit Umhang,Frau in langem Gewand mit Umhang nach links st...,"Wolff, Gustav Heinrich","Wolff, Gustav Heinrich",1925,1925,Deutschland,Deutschland,In Copyright,
1,EG1989.1.186,Sitzender weiblicher Akt,Skizze eines sitzenden weiblichen Aktes mit de...,"Wolff, Gustav Heinrich","Wolff, Gustav Heinrich",1926,1926,Deutschland,Deutschland,In Copyright,
2,O1895.610,"Prunkwagen, Blatt aus einer Folge",Blatt aus einer Folge von Prunkwagen.,"Hoppenhaupt, Johann Michael","Hoppenhaupt, Johann Michael",1750 — 1755,1750 — 1755,Berlin,Berlin,In Copyright,
3,EG2018.23.100,"Waisenhaus, Hamburg. Wettbewerbsentwurf Kernko...","Waisenhaus, Hamburg. Wettbewerbsentwurf Kernko...","Bülau, Theodor","Bülau, Theodor",1854,1854,Hamburg,Hamburg,In Copyright,
4,EG2018.23.28,"Doppelhaus, Raboisen 66-68, Hamburg. Kellerfen...","Doppelhaus, Raboisen 66-68, Hamburg. Kellerfen...","Bülau, Theodor","Bülau, Theodor",1845,1845,Hamburg,Hamburg,In Copyright,
...,...,...,...,...,...,...,...,...,...,...,...
26289,2012.163,,"Vier Korpusteile: Birne, oberes und unteres Mi...",Pask & Koenig,Pask & Koenig,1846 — 1855,1846 — 1855,London,London,In Copyright,
26290,IE1896.152,Kuzufu-Stoff: Eine Übersicht,,Nakajima Gyōzan,Nakajima Gyōzan,1872,1872,Tokio,Tokio,In Copyright,
26291,1984.496,Pelike (A: Kampf von Herakles und einer Amazon...,Bauchiges Gefäß auf auswärts gespreiztem Fuß. ...,Klasse der Rote-Linien-Peliken,Klasse der Rote-Linien-Peliken,-0510 — -0490,-0510 — -0490,Athen,Athen,In Copyright,
26292,O1911.243,Treppen,Treppen mit Grundriss. Blatt 1 (SSS 409) aus e...,"Anckermann, Bernhard Christoph","Anckermann, Bernhard Christoph",1725 — 1735,1725 — 1735,Augsburg,Augsburg,In Copyright,


In [ ]:
path = "models"
models = rq.get(base+collection_id+path).json()
models = [m["id"] for m in models]
models

In [ ]:
dmg.coll.filter("stoel")
# dmg.loc[["1999-0032_03-15"]].coll.get_texts()

In [ ]:
path = "search/order"

params = dict(object_ids="3535", concept="stoel", model_ids=",".join(models))
search_res = rq.get(base+collection_id+path, params).json()
# scores = pd.Series(search_res).sort_values()


print("\n".join(pd.DataFrame.from_records(search_res).title[:10]))

In [ ]:
score_ls = []
path = "search"
for m in models:
    print(parse_id_list(m))
    params = dict(object_ids="1999-0032_03-15", concept="", model_ids=m) #,".join(models))
    search_res = rq.get(base+collection_id+path, params).json()
    scores = pd.Series(search_res).sort_values()
    score_ls.append(scores)
    print(scores.index[-10:])


params = dict(object_ids="", concept="meubel", model_ids="") #,".join(models))
search_res = rq.get(base+collection_id+path, params).json()
scores = pd.Series(search_res).sort_values()
score_ls.append(scores)


In [ ]:
# dmg.loc[scores.sort_values()[-10:].index]
# dmg.coll.order(scores)
pd.DataFrame.from_records(search_res)



In [ ]:
dmg.loc["2003-0147_2-2"]

In [ ]:

models = rq.get("http://0.0.0.0:8080/DMG_2025-05-13/models").json()
models = [d["id"] for d in models]

# for d in range(10, 30):
#     q = dict(ISO_8601_datetime=f"2024-05-{d}", 
#              lat_long_degrees="51.05,3.71")
timings = []
for _ in tqdm(range(100)):
    q = dict(object_ids="2015-0130,1987-1343_2-4,2001-0086,3703_0-2",
                 concept="twee halskettingen, deze zitten samen in 1 doosje",
                 model_list=",".join(models))
    
    # print(base+collection_id+path)
    # print(q)
    
    t0 = time()
    
    resp = rq.get(base+collection_id+path, params=q)
        
    resp.json()
    
    # print(time() - t0)
    timings.append(time() - t0)

In [ ]:
import pandas as pd
pd.Series(timings).hist(bins=40)